In [2]:
from anomaly_detection_helper import *
from anomaly_detection_class import anomalyDetector

In [148]:
import imp
import anomaly_detection_helper
imp.reload(anomaly_detection_helper)
from anomaly_detection_helper import *

In [3]:
full_optiondata = pd.read_parquet('TSLA_2014_2021.parquet')
splitdate = '2020-08-31'
splitratio = 5
adj_full_optiondata = adjust_stock_split(full_optiondata,splitdate,splitratio)

# Train TSLA on 2014/1 -  2020/10, test on 2020/11 - 2021/5 

In [26]:
A1 = {'variable':['volume','pcratio','impliedvol'], 
        'direction':['>','<','>'], 
        'volume_operation':'sum', 
        'window':10, 
        'log_trans':False}

A2 = {'variable':['volume','pcratio','impliedvol'], 
        'direction':['>','<','>'], 
        'volume_operation':'sum', 
        'window':60, 
        'log_trans':False}

A3 = {'variable':['volume','pcratio','impliedvol'], 
        'direction':['>','<','>'], 
        'volume_operation':'sum', 
        'window':252, 
        'log_trans':False}

A4 = {'variable':['volume','pcratio','impliedvol'], 
        'direction':['>','<','>'], 
        'volume_operation':'sum', 
        'window':120, 
        'log_trans':False}

B1 = {'variable':['openinterest','pcratio','impliedvol'], 
        'direction':['>','<','>'], 
        'volume_operation':'sum', 
        'window':60, 
        'log_trans':False}

B2 = {'variable':['volume','impliedvol'], 
        'direction':['>','>'], 
        'volume_operation':'sum', 
        'window':60, 
        'log_trans':False}
B3 = {'variable':['volume','impliedvol'], 
        'direction':['>','>'], 
        'volume_operation':'sum', 
        'window':60, 
        'log_trans':True}

split_date = '2019-06-01'
win_thresh = 0

## A. Test window size 10/60/252

### A.1 window size 10

In [7]:
split_train_test = '2020-11-01'
train, test = train_test_split(adj_full_optiondata, split_train_test, window=10)
ad = anomalyDetector()
ad.compile(_type='call', expiry='next', monthly=True, moneyness='otm', params=A1)
ad.fit(train, win_thresh)

Testing 0th combo
Testing 500th combo
Testing 1000th combo
Testing 1500th combo
Testing 2000th combo


In [10]:
ad.evaluate(winprob=0.7, topn=1)

,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
2286,0.702,47,0.076,-0.043,0.040538,"[3.0, 1.3, 0.6]"
2276,0.711,38,0.076,-0.040,0.042476,"[3.0, 1.2, 0.6]"
2266,0.703,37,0.078,-0.040,0.042954,"[3.0, 1.1, 0.6]"
2278,0.706,34,0.083,-0.042,0.046250,"[3.0, 1.2, 0.8]"
1136,0.710,31,0.088,-0.025,0.055230,"[2.0, 0.8, 0.6]"
1246,0.733,30,0.088,-0.026,0.057562,"[2.1, 0.8, 0.6]"
1248,0.704,27,0.098,-0.026,0.061296,"[2.1, 0.8, 0.8]"
1355,0.720,25,0.079,-0.030,0.048480,"[2.2, 0.8, 0.5]"
2230,0.708,24,0.083,-0.032,0.049420,"[3.0, 0.8, 0.0]"
1465,0.739,23,0.082,-0.024,0.054334,"[2.3, 0.8, 0.5]"


In [11]:
ad.predict(test, win_thresh=0, param_thresh=[3.0, 1.3, 0.6], verbose=True)

+1 day stock price increase probability: 0.4
+1 day stock price avg increase: 0.055
+1 day stock price avg decrease -0.028 

+5 day stock price increase probability: 0.2
+5 day stock price avg increase: 0.23
+5 day stock price avg decrease: -0.035
+5 day expected return: 0.018000000000000002 

            outlierday  nextday  next5thday
quotedate                                  
2020-11-17      440.95   484.02      554.91
2021-01-25      872.90   886.99      837.45
2021-02-22      717.29   710.40      715.46
2021-04-26      737.02   707.27      685.35
2021-05-12      593.91   574.00      580.55


In [12]:
ad._test_performance

,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
0,0.2,5,0.23,-0.035,0.018,"[3.0, 1.3, 0.6]"


### A.2 window size 60

In [13]:
train, test = train_test_split(adj_full_optiondata, split_train_test, window=60)
ad2 = anomalyDetector()
ad2.compile(_type='call', expiry='next', monthly=True, moneyness='otm', params=A2)
ad2.fit(train, win_thresh)

Testing 0th combo
Testing 500th combo
Testing 1000th combo
Testing 1500th combo
Testing 2000th combo


In [14]:
ad2.evaluate(winprob=0.7, topn=1)

,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
405,0.705,88,0.062,-0.058,0.026600,"[1.3, 1.2, 0.5]"
733,0.701,87,0.068,-0.051,0.032419,"[1.6, 1.2, 0.3]"
645,0.709,86,0.068,-0.059,0.031043,"[1.5, 1.4, 0.5]"
635,0.723,83,0.068,-0.057,0.033375,"[1.5, 1.3, 0.5]"
843,0.707,82,0.068,-0.049,0.033719,"[1.7, 1.2, 0.3]"
...,...,...,...,...,...,...
455,0.875,8,0.118,-0.095,0.091375,"[1.4, 0.6, 0.5]"
236,0.857,7,0.131,-0.095,0.098682,"[1.2, 0.6, 0.6]"
469,0.833,6,0.058,-0.005,0.047479,"[1.4, 0.7, 0.9]"
237,0.800,5,0.121,-0.095,0.077800,"[1.2, 0.6, 0.7]"


In [16]:
ad2.predict(test, win_thresh=0, param_thresh=[1.3, 1.2, 0.5], verbose=True)

+1 day stock price increase probability: 0.571
+1 day stock price avg increase: 0.021
+1 day stock price avg decrease -0.018 

+5 day stock price increase probability: 0.857
+5 day stock price avg increase: 0.061
+5 day stock price avg decrease: -0.023
+5 day expected return: 0.048988 

            outlierday  nextday  next5thday
quotedate                                  
2020-11-25      572.91   585.15      593.64
2020-11-30      574.53   582.21      643.69
2020-12-09      606.33   624.33      622.51
2020-12-18      638.61   651.15      666.01
2021-03-26      612.70   609.20      695.05
2021-03-31      668.49   659.92      682.17
2021-05-12      593.91   574.00      580.55


In [25]:
ad2._test_performance

,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
0,0.857,7,0.061,-0.023,0.048988,"[1.3, 1.2, 0.5]"


### A.3 window size 252

In [18]:
train, test = train_test_split(adj_full_optiondata, split_train_test, window=252)
ad3 = anomalyDetector()
ad3.compile(_type='call', expiry='next', monthly=True, moneyness='otm', params=A3)
ad3.fit(train, win_thresh)
ad3.evaluate(winprob=0.7, topn=1)

Testing 0th combo
Testing 500th combo
Testing 1000th combo
Testing 1500th combo
Testing 2000th combo


,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
533,0.704,108,0.064,-0.059,0.027592,"[1.4, 1.4, 0.3]"
653,0.714,105,0.064,-0.052,0.030824,"[1.5, 1.5, 0.3]"
523,0.712,104,0.064,-0.058,0.028864,"[1.4, 1.3, 0.3]"
294,0.709,103,0.067,-0.061,0.029752,"[1.2, 1.2, 0.4]"
643,0.725,102,0.063,-0.055,0.030550,"[1.5, 1.4, 0.3]"
...,...,...,...,...,...,...
1029,1.000,6,0.069,NaN,NaN,"[1.9, 0.8, 0.9]"
1259,1.000,5,0.082,NaN,NaN,"[2.1, 0.9, 0.9]"
689,1.000,4,0.062,NaN,NaN,"[1.6, 0.7, 0.9]"
1359,1.000,3,0.095,NaN,NaN,"[2.2, 0.8, 0.9]"


In [19]:
ad3.predict(test, win_thresh=0, param_thresh=[1.4, 1.4, 0.3], verbose=True)

+1 day stock price increase probability: 0.0
+1 day stock price avg increase: nan
+1 day stock price avg decrease -0.009 

+5 day stock price increase probability: 1.0
+5 day stock price avg increase: 0.073
+5 day stock price avg decrease: nan
+5 day expected return: nan 

            outlierday  nextday  next5thday
quotedate                                  
2021-03-26      612.70   609.20      695.05
2021-03-31      668.49   659.92      682.17


In [21]:
ad3._test_performance

,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
0,1.0,2,0.073,NaN,NaN,"[1.4, 1.4, 0.3]"


### A.4 window size 120

In [20]:
train, test = train_test_split(adj_full_optiondata, split_train_test, window=120)
ad4 = anomalyDetector()
ad4.compile(_type='call', expiry='next', monthly=True, moneyness='otm', params=A4)
ad4.fit(train, win_thresh)
ad4.evaluate(winprob=0.7, topn=1)

Testing 0th combo
Testing 500th combo
Testing 1000th combo
Testing 1500th combo
Testing 2000th combo


,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
731,0.707,99,0.066,-0.045,0.033477,"[1.6, 1.2, 0.1]"
107,0.723,94,0.080,-0.060,0.041220,"[1.0, 1.5, 0.7]"
76,0.710,93,0.077,-0.066,0.035530,"[1.0, 1.2, 0.6]"
97,0.725,91,0.079,-0.062,0.040225,"[1.0, 1.4, 0.7]"
525,0.711,90,0.070,-0.060,0.032430,"[1.4, 1.3, 0.5]"
...,...,...,...,...,...,...
1469,1.000,6,0.054,NaN,NaN,"[2.3, 0.8, 0.9]"
908,1.000,5,0.055,NaN,NaN,"[1.8, 0.7, 0.8]"
909,1.000,4,0.062,NaN,NaN,"[1.8, 0.7, 0.9]"
1459,1.000,3,0.037,NaN,NaN,"[2.3, 0.7, 0.9]"


In [23]:
ad4.predict(test, win_thresh=0, param_thresh=[1.6, 1.2, 0.1], verbose=True)

+1 day stock price increase probability: 0.333
+1 day stock price avg increase: 0.036
+1 day stock price avg decrease -0.016 

+5 day stock price increase probability: 0.5
+5 day stock price avg increase: 0.121
+5 day stock price avg decrease: -0.019
+5 day expected return: 0.051 

            outlierday  nextday  next5thday
quotedate                                  
2020-12-31      704.80   729.92      875.55
2021-02-22      717.29   710.40      715.46
2021-02-23      710.40   736.74      688.60
2021-03-26      612.70   609.20      695.05
2021-03-31      668.49   659.92      682.17
2021-05-12      593.91   574.00      580.55


In [24]:
ad4._test_performance

,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
0,0.5,6,0.121,-0.019,0.051,"[1.6, 1.2, 0.1]"


## B. Test variable

### B1. replace volume with open interest

In [27]:
train, test = train_test_split(adj_full_optiondata, split_train_test, window=60)
adb1 = anomalyDetector()
adb1.compile(_type='call', expiry='next', monthly=True, moneyness='otm', params=B1)
adb1.fit(train, win_thresh)
adb1.evaluate(winprob=0.7, topn=1)

Testing 0th combo
Testing 500th combo
Testing 1000th combo
Testing 1500th combo
Testing 2000th combo


,winprob,outliernum,posret,negret,expret,openinterest/pcratio/impliedvol


In [28]:
adb1.predict(test, win_thresh=0, param_thresh=[2.1, 1.5, 0.6], verbose=True)

+1 day stock price increase probability: 0.5
+1 day stock price avg increase: 0.043
+1 day stock price avg decrease -0.024 

+5 day stock price increase probability: 1.0
+5 day stock price avg increase: 0.086
+5 day stock price avg decrease: nan
+5 day expected return: nan 

            outlierday  nextday  next5thday
quotedate                                  
2020-12-01      582.21   568.43      647.95
2020-12-02      568.43   593.64      606.33


In [29]:
adb1._test_performance

,winprob,outliernum,posret,negret,expret,openinterest/pcratio/impliedvol
0,1.0,2,0.086,NaN,NaN,"[2.1, 1.5, 0.6]"


### B2. use only volume & impliedvol

In [33]:
train, test = train_test_split(adj_full_optiondata, split_train_test, window=60)
adb2 = anomalyDetector()
adb2.compile(_type='call', expiry='next', monthly=True, moneyness='otm', params=B2)
adb2.fit(train, win_thresh)
adb2.evaluate(winprob=0.7, topn=1)

Testing 0th combo


,winprob,outliernum,posret,negret,expret,volume/impliedvol
55,0.703,91,0.069,-0.059,0.030984,"[1.5, 0.5]"
56,0.709,79,0.072,-0.055,0.035043,"[1.5, 0.6]"
75,0.705,78,0.069,-0.060,0.030945,"[1.7, 0.5]"
66,0.712,73,0.072,-0.057,0.034848,"[1.6, 0.6]"
76,0.721,68,0.073,-0.055,0.037288,"[1.7, 0.6]"
86,0.708,65,0.074,-0.055,0.036332,"[1.8, 0.6]"
67,0.705,61,0.071,-0.061,0.032060,"[1.6, 0.7]"
96,0.712,59,0.078,-0.057,0.039120,"[1.9, 0.6]"
77,0.714,56,0.072,-0.059,0.034534,"[1.7, 0.7]"
78,0.707,41,0.063,-0.060,0.026961,"[1.7, 0.8]"


In [36]:
adb2.predict(test, win_thresh=0, param_thresh=[1.5, 0.5], verbose=True)

+1 day stock price increase probability: 0.5
+1 day stock price avg increase: 0.016
+1 day stock price avg decrease -0.009 

+5 day stock price increase probability: 1.0
+5 day stock price avg increase: 0.076
+5 day stock price avg decrease: nan
+5 day expected return: nan 

            outlierday  nextday  next5thday
quotedate                                  
2020-11-30      574.53   582.21      643.69
2020-12-18      638.61   651.15      666.01
2021-03-26      612.70   609.20      695.05
2021-03-31      668.49   659.92      682.17


In [37]:
adb2._test_performance

,winprob,outliernum,posret,negret,expret,volume/impliedvol
0,1.0,4,0.076,NaN,NaN,"[1.5, 0.5]"


### B3. log transform volume

In [38]:
train, test = train_test_split(adj_full_optiondata, split_train_test, window=60)
adb3 = anomalyDetector()
adb3.compile(_type='call', expiry='next', monthly=True, moneyness='otm', params=B3)
adb3.fit(train, win_thresh)
adb3.evaluate(winprob=0.7, topn=1)

Testing 0th combo


,winprob,outliernum,posret,negret,expret,volume/impliedvol
165,0.75,8,0.064,-0.041,0.03775,"[2.6, 0.5]"
159,0.75,4,0.114,-0.056,0.07150,"[2.5, 0.9]"
169,1.00,2,0.098,NaN,NaN,"[2.6, 0.9]"


In [45]:
adb3.predict(test, win_thresh=0, param_thresh=[2.6, 0.5], verbose=True)

+1 day stock price increase probability: nan
+1 day stock price avg increase: nan
+1 day stock price avg decrease nan 

+5 day stock price increase probability: nan
+5 day stock price avg increase: nan
+5 day stock price avg decrease: nan
+5 day expected return: nan 

Empty DataFrame
Columns: [outlierday, nextday, next5thday]
Index: []


In [46]:
adb3._test_performance

,winprob,outliernum,posret,negret,expret,volume/impliedvol
0,NaN,0,NaN,NaN,NaN,"[2.6, 0.5]"
